# Spark Machine learning 

### Learning goals:
- align the relationships between Hadoop, Spark
- differentiate between Spark RDDs and Spark Dataframes and when each is appropriate
- locate and explore the Spark.ML documentation
- code along a text classification problem using four different ml algorithms, a data prep pipeline, and gridsearch to fine tune a model

# Spark context and concepts review
<img src="https://images.pexels.com/photos/285173/pexels-photo-285173.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260" width=600>

# The story of Spark (in diagrams)

## Start with Hadoop
<img alt="diagram of hadoop v1 compared to hadoop v2" src="img/yarn.png" width=800>

[diagram source](https://sites.google.com/site/codingbughunter/hadoop/yarn-general-discribe)

## Yarn facilitates the resource allocation between Spark and the HDFS
### YARN = Yet Another Resource Negotiator
#### YARN is a subproduct of Hadoop
![yarn diagram with spark](http://hortonworks.com/wp-content/uploads/2013/06/YARN.png)

[diagram source](https://sites.google.com/site/codingbughunter/hadoop/yarn-general-discribe)

## Then visualize the Spark ecosystem built on top of that

<img alt="diagram of spark eco system components" src="img/spark_eco.png" width=600>

[image source here](https://databricks.com/spark/about)

# The story of Spark (a timeline)

|<p align="left justify">Date</p>|<p align="left justify">Product</p>|<p align="left justify">Update</p>|
|:----|:-----|:-----|
| 2002 | Hadoop | <p align="left justify">Doug Cutting starts `Apache Nutch` researching sort/merge processing</p> |
| 2006 | Hadoop |  <p align="left justify">Leaves `Nutch` and joins `Yahoo`, renaming the project `Hadoop` </p>|
| 2008 | Hadoop |  <p align="left justify">`Hadoop` was made `Apache’s` top level project </p> |
| Jan 2008 | Hadoop |  <p align="left justify">v 0.10.1 released </p>|
| 2009 | Spark | <p align="left justify">started as a research project at the UC Berkeley AMPLab  </p>|
| 2010 | Spark |  <p align="left justify">open sourced </p>|
| Sept 2012 | Spark |  <p align="left justify">0.6.0 released </p>|
| 2013 | Spark |  <p align="left justify">moved to the `Apache` Software Foundation </p>|
| Feb 2013| Spark |  <p align="left justify">Spark 0.7 adds a Python API called `PySpark` </p>|
| Sept 2013 | Spark | <p align="left justify">0.8.0 introduces `MLlib` </p>|
| 2013 | Databricks |  <p align="left justify">Original Spark research team at UC Berkeley found Databricks</p> |
| May 2014 |Spark |  <p align="left justify">v 1.0 introduces Spark SQL, for loading and manipulating structured data in Spark</p>|
|Mar 2015 | Spark | <p align="left justify"> v 1.3.0 brings a new DataFrame API</p> |
| Jan 2016|  Spark | <p align="left justify"> v 1.6.0 brings a new Dataset API </p> |
| Jul 2016 | Spark | <p align="left justify"> v 2.0.0 In Scala and Java, DataFrame and Dataset have been unified, i.e. DataFrame is just a type alias for Dataset of Row. In Python and R DataFrame is the main programming interface. <br> - SparkSession: new entry point that replaces the old SQLContext<br>- Native CSV data source, based on Databricks’ spark-csv module<br>- MLlib - The DataFrame-based API is now the primary API. The RDD-based API is entering maintenance mode </p> |
| Jul 2017| Spark | <p align="left justify"> v 2.2.0 drops support for Python 2.6 |
| Nov 2018 | Spark | <p align="left justify"> v 2.4.0<br> - This release adds Barrier Execution Mode for better integration with deep learning frameworks<br> - more integration between pandas UDF and spark DataFrames </p>|
| Jan 2020| PySpark | <p align="left justify"> Python 2 will be officially unsupported |
    

# Spark data objects

<img alt="diagram of definitions of Spark objects from databricks" src="https://databricks.com/wp-content/uploads/2018/05/rdd-1024x595.png" width=700>

## Differences between objects:
<img alt="memory usage" src="https://databricks.com/wp-content/uploads/2016/07/memory-usage-when-caching-datasets-vs-rdds.png" width=600>

<img alt="type safety" src="https://databricks.com/wp-content/uploads/2016/07/sql-vs-dataframes-vs-datasets-type-safety-spectrum.png" width=600>

### Use an RDD when:
[quoted from databricks](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html)

> - you want low-level transformation and actions and control on your dataset;
> - your data is unstructured, such as media streams or streams of text;
> - you want to manipulate your data with functional programming constructs than domain specific expressions;
> - you don’t care about imposing a schema, such as columnar format, while processing or accessing data attributes by name or column

### Use a dataframe when:
[also quoted from databricks](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html)


> - you want rich semantics, high-level abstractions, and domain specific APIs, use DataFrame
> - your processing demands high-level expressions, filters, maps, aggregation, averages, sum, SQL queries, columnar access and use of lambda functions on semi-structured data, use DataFrame
> - you want higher degree of type-safety at compile time, want typed JVM objects, take advantage of Catalyst optimization, and benefit from Tungsten’s efficient code generation, use Dataset.
> - you want unification and simplification of APIs across Spark Libraries, use DataFrame or Dataset.
> - If you are a R user, use DataFrames.
> - If you are a Python user, use DataFrames and resort back to RDDs if you need more control.

**Note**: Machine learning algorithms are run on _DataFrames_

## Review:

- You are grabbing live tweets about the CW show 'Jane the Virgin' for later analysis. In the Spark ecosystem, where should you store them? an RDD or a DataFrame?

- You have an RDD of data that you wish to use to build a predictive model. Should you leave it as an RDD or transform it to a DataFrame?


## Step 1: Create spark context

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
spark

## Step 2: Create a pandas dataframe

In [3]:
!head data/users.csv

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


In [6]:
import pandas as pd
p_df = pd.read_csv('data/users.csv', sep='|', header=None, names=['id',
                                                                  'age',
                                                                  'gender',
                                                                  'role',
                                                                  'zip'])

In [7]:
p_df

,id,age,gender,role,zip
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


## Step 3: Create spark dataframe

In [8]:
rdd = sc.textFile("data/users.csv").map(lambda line: line.split("|"))

In [9]:
rdd.take(5)

[['1', '24', 'M', 'technician', '85711'],
 ['2', '53', 'F', 'other', '94043'],
 ['3', '23', 'M', 'writer', '32067'],
 ['4', '24', 'M', 'technician', '43537'],
 ['5', '33', 'F', 'other', '15213']]

In [10]:
df = spark.read.csv("data/users.csv", sep="|")

In [11]:
df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string]

In [12]:
# alternatively:
from_pandas_df = spark.createDataFrame(p_df)

In [13]:
from_pandas_df

DataFrame[id: bigint, age: bigint, gender: string, role: string, zip: string]

In [14]:
df.take(5)

[Row(_c0='1', _c1='24', _c2='M', _c3='technician', _c4='85711'),
 Row(_c0='2', _c1='53', _c2='F', _c3='other', _c4='94043'),
 Row(_c0='3', _c1='23', _c2='M', _c3='writer', _c4='32067'),
 Row(_c0='4', _c1='24', _c2='M', _c3='technician', _c4='43537'),
 Row(_c0='5', _c1='33', _c2='F', _c3='other', _c4='15213')]

In [15]:
from_pandas_df.take(5)

[Row(id=1, age=24, gender='M', role='technician', zip='85711'),
 Row(id=2, age=53, gender='F', role='other', zip='94043'),
 Row(id=3, age=23, gender='M', role='writer', zip='32067'),
 Row(id=4, age=24, gender='M', role='technician', zip='43537'),
 Row(id=5, age=33, gender='F', role='other', zip='15213')]

In [16]:
# it's already a DF, but this is the easy way to rename columns
df = (spark.read.csv("data/users.csv", sep="|")
           .toDF("id", "age", "gender", "occupation", "zip"))

In [17]:
df

DataFrame[id: string, age: string, gender: string, occupation: string, zip: string]

In [32]:
results = (
    df.where("occupation != 'other'")
      .groupby("occupation")
      .count()
      .sort("count", ascending=0)
)

In [34]:
results.toPandas()

,occupation,count
0,student,196
1,educator,95
2,administrator,79
3,engineer,67
4,programmer,66
5,librarian,51
6,writer,45
7,executive,32
8,scientist,31
9,artist,28


In [24]:
df.persist()

DataFrame[id: string, age: string, gender: string, occupation: string, zip: string]

In [38]:
# type(df.limit(5))
# df.take(5).toPandas()
type(df.show(5))

+---+---+------+----------+-----+
| id|age|gender|occupation|  zip|
+---+---+------+----------+-----+
|  1| 24|     M|technician|85711|
|  2| 53|     F|     other|94043|
|  3| 23|     M|    writer|32067|
|  4| 24|     M|technician|43537|
|  5| 33|     F|     other|15213|
+---+---+------+----------+-----+
only showing top 5 rows



NoneType

In [ ]:
import pyspark.sql.functions as F
df.agg(F.countDistinct('occupation')).show()

In [41]:
# We can also use SQL to query
df.createOrReplaceTempView('users')
results.createOrReplaceTempView('occupation_count')
query = """
SELECT *
FROM users u
JOIN occupation_count oc ON oc.occupation = u.occupation
LIMIT 10
"""
output = spark.sql(query)
output.show(10)

+---+---+------+-------------+-----+-------------+-----+
| id|age|gender|   occupation|  zip|   occupation|count|
+---+---+------+-------------+-----+-------------+-----+
|  1| 24|     M|   technician|85711|   technician|   27|
|  3| 23|     M|       writer|32067|       writer|   45|
|  4| 24|     M|   technician|43537|   technician|   27|
|  6| 42|     M|    executive|98101|    executive|   32|
|  7| 57|     M|administrator|91344|administrator|   79|
|  8| 36|     M|administrator|05201|administrator|   79|
|  9| 29|     M|      student|01002|      student|  196|
| 10| 53|     M|       lawyer|90703|       lawyer|   12|
| 13| 47|     M|     educator|29206|     educator|   95|
| 14| 45|     M|    scientist|55106|    scientist|   31|
+---+---+------+-------------+-----+-------------+-----+

